In [3]:
#from nba_api.stats.static import players
#
#
# players.find_players_by_full_name("Lebron James")
#
#players.find_player_by_id(2544)

In [2]:
#Module load
from nba_api.stats.endpoints import commonallplayers #per avere l'elenco dei giocatori (id)
from nba_api.stats.endpoints import playercareerstats #per avere i dati
from nba_api.stats.endpoints import playerawards #per sapere chi ha vinto MVP
from nba_api.stats.endpoints import teamyearbyyearstats #classifiche finali delle squadre
import pandas as pd
from tqdm import tqdm
import pickle

In [3]:
from_year = 2010

total_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]
total_players['TO_YEAR'] = total_players['TO_YEAR'].astype('int')
total_players = total_players[total_players['TO_YEAR'] > from_year]

#Ho controllato che in total_players, il PERSON_ID sia univoco
total_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
19,1629121,"Adams, Jaylen",Jaylen Adams,0,2018,2020,jaylen_adams,jaylen_adams,0,,,,,None,Y,01
20,203919,"Adams, Jordan",Jordan Adams,0,2014,2015,jordan_adams,jordan_adams,0,,,,,None,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,203092,"Zeller, Tyler",Tyler Zeller,0,2012,2019,tyler_zeller,tyler_zeller,0,,,,,None,Y,00
4720,1627757,"Zimmerman, Stephen",Stephen Zimmerman,0,2016,2016,stephen_zimmerman,stephen_zimmerman,0,,,,,None,Y,01
4721,1627835,"Zipser, Paul",Paul Zipser,0,2016,2017,paul_zipser,paul_zipser,0,,,,,None,Y,01
4722,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01


In [40]:
# total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
# total_stats = pd.DataFrame()

In [41]:
# stats = playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]
# stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)
# awards = playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]
# awards = awards.drop(awards.columns[[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1)
# awards = awards.rename(columns={"SEASON": "SEASON_ID"})
# awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"})
# awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

In [4]:
all_teams_history = list(total_players["TEAM_ID"].drop_duplicates())
data_all_team_stats = pd.DataFrame()

for team in all_teams_history:
    t = teamyearbyyearstats.TeamYearByYearStats(team_id=team).get_data_frames()[0]
    data_all_team_stats = pd.concat([data_all_team_stats, t], axis=0, ignore_index=True)

#data_all_team_stats = [int("20" + x.split("-")[1]) for x in all_teams_history] NON SERVE, FACCIO DOPO -- ELIMINARE

data_all_team_stats = data_all_team_stats.rename(columns={"YEAR": "SEASON_ID"})
data_all_team_stats = data_all_team_stats[["TEAM_ID", "SEASON_ID", "CONF_RANK"]]

with open("data_all_team_stats.pkl",'wb') as f:
    pickle.dump(data_all_team_stats,f)

data_all_team_stats


,TEAM_ID,SEASON_ID,CONF_RANK
0,1610612761,1995-96,14
1,1610612761,1996-97,12
2,1610612761,1997-98,15
3,1610612761,1998-99,10
4,1610612761,1999-00,6
...,...,...,...
1562,1610612742,2017-18,13
1563,1610612742,2018-19,14
1564,1610612742,2019-20,7
1565,1610612742,2020-21,5


In [5]:
total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
total_stats = pd.DataFrame()
with open("data_all_team_stats.pkl", "rb") as f:
    data_all_team_stats = pickle.load(f)

for index, row in tqdm(total_names.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]

    stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)

    awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]

    awards = awards.drop(awards.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1) #Tolgo anche PERSON_ID
    awards = awards.rename(columns={"SEASON": "SEASON_ID"})
    #awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"}) #tolto perché dava interferenza
    awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

    #merge
    stats=stats.merge(awards, how="left", on="SEASON_ID")

    #1 to MVP, 0 to other
    stats["DESCRIPTION"] = stats["DESCRIPTION"].replace("NBA Most Valuable Player", 1) #Replace the description MVP with 1
    stats["DESCRIPTION"] = stats["DESCRIPTION"].fillna(0) #Replace NULL with 0
    stats = stats.rename(columns={"DESCRIPTION": "MVP"})

    #merge della classifica in cui è arrivata la squadra in una certa stagione
    stats = stats.merge(data_all_team_stats, how="inner", on=["SEASON_ID", "TEAM_ID"])

    total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)


#Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])] #Problema se si prendono dati < anno 2000
#Prendo i dati superiore all'anno FROM_YEAR
total_stats= total_stats[total_stats["SEASON_ID"] > from_year]


with open("player_stats.pkl",'wb') as f:
    pickle.dump(total_stats,f)


7it [00:34,  4.99s/it]


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [57]:

#Find lebron james totals stats
#playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]

total_stats

,PLAYER_ID,SEASON_ID,TEAM_ID,PLAYER_AGE,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,DESCRIPTION,CONF_RANK
0,203518,2017,1610612760,23.0,68,1055.0,134,341,0.393,94,...,68,86,40,37,8,33,114,406,NaN,6
1,203518,2018,1610612760,24.0,75,1134.0,115,291,0.395,84,...,88,114,28,38,8,25,124,353,NaN,4
2,203518,2019,1610612760,25.0,31,588.0,56,157,0.357,41,...,43,48,20,17,6,14,53,165,NaN,6
3,1630173,2021,1610612748,21.0,61,737.0,124,228,0.544,0,...,135,208,29,20,28,43,91,304,NaN,6
4,1630173,2022,1610612761,22.0,26,641.0,85,214,0.397,9,...,138,198,40,14,14,30,57,207,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8920,1627826,2019,1610612747,22.0,33,516.0,112,193,0.580,0,...,108,162,25,4,27,33,73,281,NaN,10
8921,1627826,2019,1610612746,22.0,26,524.0,100,186,0.538,0,...,139,200,38,10,24,37,64,244,NaN,8
8922,1627826,2020,1610612746,23.0,72,1326.0,236,385,0.613,0,...,346,543,82,16,66,61,168,596,NaN,2
8923,1627826,2021,1610612746,24.0,72,1609.0,257,394,0.652,1,...,330,519,90,24,62,81,187,650,NaN,4


#How it's structured this dataset
- Player_id --> id del giocatore(da ritrovare il nome con players.find_player_by_id())
- Season_id --> id della stagione
- ...other...
- GP --> Games Played
- GS --> Games ... (idk but similar to GP) (I think is not usefull)
- MIN Minutes Played
- FGM Field Goals Made --> Numero canestri fatti
- FGA Field Goals Attempted --> Numero tiri provati
- FG% Field Goal Percentage --> FGM/FGA, rapporto precisione tiri
- 3PM 3 Point Field Goals Made --> Numero canestri da 3 fatti
- 3PA 3 Point Field Goals Attempted --> Numero tiri da 3 provati
- 3P% 3 Point Field Goals Percentage --> 3PM/3PA, precisione tiri da 3
- FTM Free Throws Made --> Tiri liberi fatti
- FTA Free Throws Attempted --> tiri liberi provati
- FT% Free Throw Percentage --> FTM/FTA
- OREB Offensive Rebounds --> Rimbalzi conquistati in attacco
- DREB Defensive Rebounds --> rimbalzi conquistati in difesa
- REB Rebounds --> OREB + DREB, rimbalzi totali
- AST Assists
- TOV Turnovers --> palla persa
- STL Steals --> palla rubata
- BLK Blocks --> blocchi
- PF Personal Fouls --> falli personali
- DD2 Double Doubles --> doppie doppie
- TD3 Trible Doubles --> triple doppie
- PTS Points

In [6]:
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)

total_stats





,PLAYER_ID,SEASON_ID,TEAM_ID,PLAYER_AGE,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,DESCRIPTION,CONF_RANK
0,203518,2017,1610612760,23.0,68,1055.0,134,341,0.393,94,...,68,86,40,37,8,33,114,406,NaN,6
1,203518,2018,1610612760,24.0,75,1134.0,115,291,0.395,84,...,88,114,28,38,8,25,124,353,NaN,4
2,203518,2019,1610612760,25.0,31,588.0,56,157,0.357,41,...,43,48,20,17,6,14,53,165,NaN,6
3,1630173,2021,1610612748,21.0,61,737.0,124,228,0.544,0,...,135,208,29,20,28,43,91,304,NaN,6
4,1630173,2022,1610612761,22.0,26,641.0,85,214,0.397,9,...,138,198,40,14,14,30,57,207,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8920,1627826,2019,1610612747,22.0,33,516.0,112,193,0.580,0,...,108,162,25,4,27,33,73,281,NaN,10
8921,1627826,2019,1610612746,22.0,26,524.0,100,186,0.538,0,...,139,200,38,10,24,37,64,244,NaN,8
8922,1627826,2020,1610612746,23.0,72,1326.0,236,385,0.613,0,...,346,543,82,16,66,61,168,596,NaN,2
8923,1627826,2021,1610612746,24.0,72,1609.0,257,394,0.652,1,...,330,519,90,24,62,81,187,650,NaN,4


In [8]:
from nba_api.stats.endpoints import BoxScoreAdvancedV2

BoxScoreAdvancedV2.player_stats().get_data_frames()[0]

TypeError: 'NoneType' object is not callable

In [22]:
from nba_api.stats.endpoints import playerawards

playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]

,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2008-09,None,None,1610612739,Award,Kia Motors,KIADT,None
1,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2009-10,None,None,1610612739,Award,Kia Motors,KIADT,None
2,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2010-11,None,None,1610612748,Award,Kia Motors,KIADT,None
3,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2011-12,None,None,1610612748,Award,Kia Motors,KIADT,None
4,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2012-13,None,None,1610612748,Award,Kia Motors,KIADT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Most Valuable Player of the ...,None,2005-06,None,None,None,Award,Sporting News,SNMVP,None
145,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Rookie of the Year,None,2003-04,None,None,None,Award,Sporting News,SNROY,None
146,2544,LeBron,James,USA,Olympic Bronze Medal,None,2004,None,None,None,Award,Olympic,Bronze,None
147,2544,LeBron,James,USA,Olympic Gold Medal,None,2008,None,None,None,Award,Olympic,Gold,None
